In [45]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [46]:
#*********************************************************************Beginning of  of Part 1 **************************************************************************************************
import urllib.request

# specify the url
quote_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
        
# query the website and return the html to the variable ‘page’
response = urllib.request.urlopen(quote_page)
page= response.read()
soup = BeautifulSoup(page, 'html.parser')


In [47]:
# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighbourhood'] 

# instantiate the dataframe
torontodf = pd.DataFrame(columns=column_names)
torontodf

#Get table
table = soup.find('table',{'class':'wikitable sortable'})

#Extract Data from Table
row_marker = 0
Postcode=""
Borough=""
Neighbourhood=""
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    
    for column in columns:
        if column_marker==0:
            Postcode=column.get_text()
        elif column_marker==1:
            Borough=column.get_text()
        elif column_marker==2:    
            Neighbourhood=column.get_text()
        column_marker += 1
      
    torontodf = torontodf.append({'Postcode':Postcode,'Borough':Borough,'Neighbourhood':Neighbourhood}, ignore_index=True)
    row_marker +=1


#Remove first Row as it is empty
torontodf = torontodf.iloc[1:]

#Remove \n from the Neighbourhood column
torontodf['Neighbourhood'] = torontodf['Neighbourhood'].str[:-1]

#Remove rows where Borough is Not assigned
torontodf = torontodf[torontodf.Borough != "Not assigned"]

#Reset Index
torontodf=torontodf.reset_index(drop=True)

#grouping  neighbourhoods by postal code
torontodf=torontodf.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

#Assigning "Not assigned" Neighbourhood values to Borough
for index, row in torontodf.iterrows():
    if row['Neighbourhood'] =="Not assigned":
        row['Neighbourhood']=row['Borough']


torontodf.head(5)

#*********************************************************************End of  of Part 1 **************************************************************************************************

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
torontodf.shape

(103, 3)

In [48]:
#*********************************************************************Beginning of  of Part 2 **************************************************************************************************

# the latitude and the longitude coordinates of each neighborhood

path="http://cocl.us/Geospatial_data"

#Load the data
Geospatial_data = pd.read_csv(path,header=None)

#Remove the first row as it has the column headers
Geospatial_data = Geospatial_data.iloc[1:]

#Rename the column headers with the correct name
Geospatial_data.columns = ['Postal Code', 'Latitude', 'Longitude']

#Reset Index
Geospatial_data.reset_index(drop = True, inplace = True)
Geospatial_data.head(3)

,Postal Code,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115


In [133]:
#Adding new column in the toronto frame for latitude
torontodf["Latitude"]=Geospatial_data["Latitude"]

#Adding new column in the toronto frame for Longitude
torontodf["Longitude"]=Geospatial_data["Longitude"]

#Change type of Latitude Column to float
torontodf.Latitude = torontodf.Latitude.astype(float)

#Change type of Longitude Column to float
torontodf.Longitude = torontodf.Longitude.astype(float)

torontodf.head(2)
#*********************************************************************End of  of Part 2 **************************************************************************************************

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497


In [50]:
# Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location= geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [38]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontodf['Latitude'], torontodf['Longitude'], torontodf['Borough'], torontodf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [134]:
#*********************************************************************Beginning of  of Part 3 **************************************************************************************************
#Exploring data in Scarborough
York_data = torontodf[torontodf['Borough'] == 'York'].reset_index(drop=True)
York_data.head(2220)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
2,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
3,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [121]:
CLIENT_ID = '1TITQDK2R10KCFRR3QVPRYQMHPKLTLYJZBXLJM1RP2IGRUVR' # your Foursquare ID
CLIENT_SECRET = 'LYDRUEYKILEJ04K55J5O2N2ZL4T4VV12HYVJPNZD4CMBW2KO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



24

In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [135]:
York_venues = getNearbyVenues(names=York_data['Neighbourhood'],
                                   latitudes=York_data['Latitude'],
                                   longitudes=York_data['Longitude']
                                  )


York_venues.head(3)

len(York_venues['Neighborhood'].unique())

5

In [136]:
# one hot encoding
York_onehot = pd.get_dummies(York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
York_onehot['Neighborhood'] = York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [York_onehot.columns[-1]] + list(York_onehot.columns[:-1])
York_onehot = York_onehot[fixed_columns]

York_onehot.head(2)

,Neighborhood,Bakery,Bus Line,Convenience Store,Fast Food Restaurant,Field,Fried Chicken Joint,Grocery Store,Hockey Arena,Market,Park,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Trail,Women's Store
0,Humewood-Cedarvale,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,Humewood-Cedarvale,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [137]:
York_grouped = York_onehot.groupby('Neighborhood').mean().reset_index()
York_grouped.head(2)
York_grouped.shape

(5, 17)

In [139]:
# set number of clusters
kclusters = 5

York_grouped_clustering = York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 


array([3, 4, 2, 1, 0], dtype=int32)

In [143]:
# add clustering labels

York_merged = York_data

York_merged.insert(0, 'Cluster Labels', kmeans.labels_)


York_merged.head(2)

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,3,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
1,4,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512


In [145]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(York_merged['Latitude'], York_merged['Longitude'], York_merged['Neighbourhood'], York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters